In [3]:
import os
import joblib
import jieba
import pandas as pd
import re
import numpy as np
from tqdm import tqdm

# 数据预处理

## 数据读入

In [162]:
# 设置要读取的文件夹路径和要转换为列表的行号（从0开始计数）
folder_path = './message'
row_index = '详细内容'

# 初始化一个空列表，用于存储所有CSV文件的指定行
data_raw = []
# 遍历文件夹中的所有CSV文件
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        # 使用pandas读取CSV文件，并将指定行转换为列表
        df = pd.read_csv(file_path)
        row_list = df[row_index].tolist()
        # 将该文件的指定行列表添加到结果列表中
        data_raw.extend(row_list)

print("一共有：" + str(len(data_raw)) + "条文本")

一共有：768条文本


## 删去停用词以及对文本进行分词

In [163]:
data_words = []
#只保留汉字
for i in tqdm(data_raw):
    if not i or str(i)=='nan':
        continue
    out_str = ''
    for word in i:
        if re.compile(r"[\u4e00-\u9fa5]").findall(word):
            out_str += word
    data_words.append(out_str)
print('Frist process compeleted')
#记载自定义词典
#jieba.load_userdict(r"./dict/owner_dict.txt")
#分词
cut_data = []
for doc in tqdm(data_words):
    document3_cut = jieba.cut(doc,use_paddle=True)
    cut_data.append(' '.join(document3_cut))
print('cut sentence compeleted')
#去停用词
data = []
index_data = []
with open('./config/hit_stopwords.txt',encoding = "utf-8") as f:
    stopwords = f.read()
    stopwords = stopwords.strip().split('\n')
for i in tqdm(cut_data):
    words = i.split(' ')
    for word in words[::-1]:
        if word in stopwords or len(word)<2:
            words.remove(word)
    if len(words) > 1:
        data.append(' '.join(words))
print('pretreatment compeleted')

100%|██████████████████████████████████████████████████████████████████████████████| 768/768 [00:00<00:00, 5086.21it/s]


Frist process compeleted


100%|██████████████████████████████████████████████████████████████████████████████| 768/768 [00:00<00:00, 1517.70it/s]


cut sentence compeleted


100%|██████████████████████████████████████████████████████████████████████████████| 768/768 [00:00<00:00, 1000.00it/s]

pretreatment compeleted


# Bert_LDA

## 预训练模型加载

In [2]:
from gensim.models import KeyedVectors
txt_file_path = r"D:\gitee\emotive_classification\ipynb\config\tencent-embedding-s.txt"
chinese_models = KeyedVectors.load_word2vec_format(txt_file_path, unicode_errors='ignore', binary=False)
chinese_models

## 模型搭建

In [1]:
from bertopic import BERTopic

In [167]:
del topic_model
topic_model = BERTopic(language="chinese (simplified)", 
                       embedding_model=chinese_models,
                       calculate_probabilities=True,
                       verbose=True)

In [168]:
topics, probs = topic_model.fit_transform(data)

100%|██████████████████████████████████████████████████████████████████████████████| 768/768 [00:00<00:00, 3657.13it/s]
2023-04-26 16:35:23,368 - BERTopic - Transformed documents to Embeddings
2023-04-26 16:35:26,238 - BERTopic - Reduced dimensionality
2023-04-26 16:35:26,311 - BERTopic - Clustered reduced embeddings


## 结果可视化

In [169]:
topic_model.visualize_topics() 

In [170]:
#结果保存
topic_result = topic_model.get_topic_info()
for index,content in topic_result.iterrows():
    keywords_str = ''
    for j in topic_model.get_topic(int(content[0])):
        if j[0] != '':
            keywords_str = keywords_str + j[0] + '_'
    topic_result.loc[index,'Name'] = keywords_str[:-1]
#topic_result.to_csv('./output/keywords.csv')
topic_result

,Topic,Count,Name
0,-1,119,小区_没有_建议_物业_领导_目前_严重_居民_问题_希望
1,0,260,道路_望京_建议_交通_地区_居民_车辆_规划_出行_行人
2,1,105,小区_改造_居民_业主_领导_希望_老旧_进行_拆迁_没有
3,2,77,退费_退款_公司_报名_北京_机构_教育_课程_有限公司_合同
4,3,50,影响_噪音_居民_扰民_严重_住户_施工_领导_垃圾桶_小区
5,4,49,孩子_小学_学校_家长_政策_学区_上学_学生_入学_校区
6,5,30,朝阳区_公司_注册_市场_落户_住宅_优士阁_监管局_产权_政策
7,6,28,车位_停车_固定_车辆_居委会_收费_小区_停车场_业主_没有
8,7,20,充电_电动车_安装_新能源_社区_小区_宽带_物业_车位_公共
9,8,15,检测_核酸_防疫_市民_采样_健康_信息_拭子_需要_屋子


In [171]:
topic_model.visualize_heatmap(n_clusters=9) 

In [172]:
topic_model.visualize_hierarchy(top_n_topics=9)

# topic2vec

In [134]:
from IPython.display import display
from top2vec import Top2Vec

### Top2vec(documents, min_count, speed, workers)

documents: 文档列表

min_count: 词语最少出现次数。低于min_count的词不加入模型中

speed: 训练速度，参数默认"learn"；“fast-learn” 速度最快，训练效果最差；“learn” 速度，训练效果中等；“deep-learn” 速度最慢，训练效果最佳

workers: 并行运行数，该值最大取值为电脑CPU的核数

In [164]:
top2vec_model = Top2Vec(documents=data, 
                min_count=5,
                speed="deep-learn",  
                workers=8)

2023-04-26 16:34:11,289 - top2vec - INFO - Pre-processing documents for training
2023-04-26 16:34:11,364 - top2vec - INFO - Creating joint document/word embedding
2023-04-26 16:34:33,679 - top2vec - INFO - Creating lower dimension embedding of documents
2023-04-26 16:34:36,913 - top2vec - INFO - Finding dense areas of documents
2023-04-26 16:34:36,941 - top2vec - INFO - Finding topics


In [165]:
# 话题个数
top2vec_model.get_num_topics()

7

In [159]:
# 各话题数量
topic_sizes, topic_nums = top2vec_model.get_topic_sizes()

{"topic_sizes":topic_sizes, "topic_ids":topic_nums}

{'topic_sizes': array([211,  84,  58], dtype=int64),
 'topic_ids': array([0, 1, 2], dtype=int64)}

In [160]:
def gen_wordcloud(topic_words, word_scores, topic_id):
    """
    topic_words: 主题词列表
    word_scores: 主题特征词的权重得分(词语表征主题的能力)
    topic_id: 主题id
    """
    import pyecharts.options as opts
    from pyecharts.charts import WordCloud
    from IPython.display import display
    
    data = [(word, str(weight)) for word, weight in zip(topic_words, word_scores)]

    wc = WordCloud()
    wc.add(series_name="", data_pair=data, word_size_range=[6, 88])
    wc.set_global_opts(
        title_opts=opts.TitleOpts(title="Topic_{topic_id}".format(topic_id=topic_id), 
                                  title_textstyle_opts=opts.TextStyleOpts(font_size=23)),
        tooltip_opts=opts.TooltipOpts(is_show=True))

    display(wc.render_notebook())

In [166]:
num = top2vec_model.get_num_topics()
topic_wordss, word_scoress, topic_ids = top2vec_model.get_topics(num)

for topic_words, word_scores, topic_id in zip(topic_wordss, word_scoress, topic_ids):
    gen_wordcloud(topic_words, word_scores, topic_id)